In [1]:
using Plots;plotly()
using Optim
using Zygote
using Meteor.Utility
using Meteor.ExactDiag
using DataFrames;using CSV
using Pkg

In [2]:
using LinearAlgebra
ccall((:openblas_get_num_threads64_, Base.libblas_name),Cint,())
LinearAlgebra.BLAS.set_num_threads(8)

In [3]:
using Base.Threads;nthreads()

7

In [4]:
function thermal_state(dims,nbar)
    c::Array{Complex{Float64}}=zeros(dims,dims)
    if nbar==0
        c[1,1]=1
    else
        for i in 1:dims
           c[i,i]= exp(-i*log(1/nbar+1))
        end
    end
    c=c/tr(c)
    return c        
end

function fock_state(dims,n)
    c::Matrix{ComplexF64}=zeros(dims,dims)
    if n<dims
        c[n+1,n+1]=1
    end
    return c
end

function eye(n)
    k::Array{Complex{Float64}}=zeros(n,n)
    for i in 1:n
        k[i,i]=1
    end
    return k    
end


function a₋(n)      #n为维数
    s::Array{Complex{Float64}}=zeros(n,n)
    for i=1:n-1
        s[i,i+1]=sqrt(i)
    end
    return s
end

a₊(n)=adjoint(a₋(n));

dim=10
ν=1
dot_num=30
initial_state=kron([1.0 0 0;0 0 0;0 0 0],thermal_state(dim,1))
#########密度矩阵向量化，超算子维数变成n^2*n^2维,态矩阵厄米，只用存一半？意义不大
function dm2vec(dm)#态向量化
#julia里是按列reshape的,reshape成列向量
    return reshape(dm,:,1)
end

function vec2dm(vec)
    return reshape(vec,3*dim,:)
end


n=kron(eye(3),a₊(dim)*a₋(dim))
en=kron([0 0 0;0 0 0;0 0 1],eye(dim))
rr=kron([0 0 0;0 1 0;0 0 0],eye(dim))
η1=0.15
η2=-0.15
a=a₊(dim)+a₋(dim)
ag2=kron([0 0 0;0 0 0;1 0 0],exp(im*η1*a))
ag3=kron([0 0 1;0 0 0;0 0 0],exp(-im*η1*a))
ag4=ag2+ag3
ar2=kron([0 0 0;0 0 0;0 1 0],exp(im*η2*a))
ar3=kron([0 0 0;0 0 1;0 0 0],exp(-im*η2*a))
ar4=ar2+ar3

E=eye(3*dim)
γ1=40/3*ν
γ2=20/3*ν

function diss1()
    dtheta = 0.01
    x=zeros((3*dim)^2,(3*dim)^2)
    for theta in -1:dtheta:1
        c = dtheta * 0.75 * (1 + theta^2)
        jump = sqrt(c / 2) * exp(im*η1*theta*a)
        aup=kron([0 0 0;0 0 0;1 0 0],jump')
        adown=kron([0 0 1;0 0 0;0 0 0],jump)
        x=x+kron(transpose(aup),adown)-0.5*kron(E,aup*adown)-0.5*kron(transpose(aup*adown),E)
    end
    return x
end
function diss2()
    dtheta = 0.01
    x=zeros((3*dim)^2,(3*dim)^2)
    for theta in -1:dtheta:1
        c = dtheta * 0.75 * (1 + theta^2)
        jump = sqrt(c / 2) * exp(im*η2*theta*a)
        aup=kron([0 0 0;0 0 0;0 1 0],jump')
        adown=kron([0 0 0;0 0 1;0 0 0],jump)
        x=x+kron(transpose(aup),adown)-0.5*kron(E,aup*adown)-0.5*kron(transpose(aup*adown),E)
    end
    return x
end

dissipation1=γ1*diss1()
dissipation2=γ2*diss2()

# Δg=(Ωr^2+Ωg^2)/ν-ν
# Δr=Δg

function eit_Lindbladian_vec(Ωg,Ωr,Δ)
    unitary::Matrix{ComplexF64}=kron(E,-im*(ν*n-Δ*en+Ωg/2*ag4+Ωr/2*ar4))+kron(transpose(im*((ν*n-Δ*en+Ωg/2*ag4+Ωr/2*ar4))),E)
    
    return unitary+dissipation1+dissipation2
end

instate=dm2vec(initial_state);
#分段优化的话应该优化比如3次，只不过改变初始态，EIT冷却过程中的态是什么样的呢
# function feval(x)
#     f=exp(50*eit_Lindbladian_vec(x[1],x[2],x[3]))*instate
#     a=real(tr(n*vec2dm(f)))

#     print(a," ")
#     return a
# end

# function g!(G,seq)
#     print("g!  ")
#     a=real(gradient(feval,seq)[1])
#     for i in 1:length(seq)
#        G[i]=a[i] 
#     end
# end

g! (generic function with 1 method)

In [5]:
# @time res=optimize(feval,g!,[3.6103223565556357,21.05702368447422,111.9645147569779],LBFGS(),Optim.Options(x_tol=1e-4))

In [6]:
# Optim.minimum(res)

In [7]:
# se1=Optim.minimizer(res)

In [17]:
methods(ptrace)

# 14 methods for generic function "ptrace":
[1] ptrace(basis::Array{Int64,1}, v::Union{AbstractArray{T,2} where T, AbstractArray{T,1} where T}, sites::Array{Int64,1}; mode) in Meteor.ExactDiag.Model.Utility at /Users/pidan/Downloads/Meteor.jl-develop/src/ExactDiag/Model/Utility/ptrace.jl:38
[2] ptrace(dim::Array{Int64,1}, m::MappedMatrix{var"#s177",1} where var"#s177"<:(AbstractArray{T,2} where T), sites::Array{Int64,1}; mode) in Meteor.ExactDiag.Model.Utility at /Users/pidan/Downloads/Meteor.jl-develop/src/ExactDiag/Model/Utility/ptrace.jl:76
[3] ptrace(dim::Array{Int64,1}, v::Meteor.ExactDiag.Basis.AbstractMappedVector, sites::Array{Int64,1}; mode) in Meteor.ExactDiag.Model.Utility at /Users/pidan/Downloads/Meteor.jl-develop/src/ExactDiag/Model/Utility/ptrace.jl:78
[4] ptrace(mps::Meteor.TensorNetwork.AbstractMPS{Array{T,3}} where T, sites::Array{Int64,1}; isunitary) in Meteor.Model.Utility at /Users/pidan/Downloads/Meteor.jl-develop/src/Model/Utility/partialtrace.jl:158
[5] ptrace(mps::Meteor.TensorNetwork.AbstractMPS, sites::Array{Int64,1}, idens; isunitary) in Meteor.Model.Utility at /Users/pidan/Downloads/Meteor.jl-develop/src/Model/Utility/partialtrace.jl:151
[6] ptrace(model::Meteor.Model.AbstractUnitaryModel, sites::Array{Int64,1}) in Meteor.Model.Utility at /Users/pidan/Downloads/Meteor.jl-develop/src/Model/Utility/partialtrace.jl:166
[7] ptrace(model::Meteor.Model.AbstractOpenModel, sites::Array{Int64,1}) in Meteor.Model.Utility at /Users/pidan/Downloads/Meteor.jl-develop/src/Model/Utility/partialtrace.jl:176
[8] ptrace(basis::Meteor.ExactDiag.Basis.AbstractNonSymmetricBasis, v::AbstractArray{T,2} where T, sites::Array{Int64,1}) in Meteor.ExactDiag.Model.Utility at /Users/pidan/Downloads/Meteor.jl-develop/src/ExactDiag/Model/Utility/ptrace.jl:34
[9] ptrace(basis::Meteor.ExactDiag.Basis.AbstractNonSymmetricBasis, v::AbstractArray{T,1} where T, sites::Array{Int64,1}) in Meteor.ExactDiag.Model.Utility at /Users/pidan/Downloads/Meteor.jl-develop/src/ExactDiag/Model/Utility/ptrace.jl:35
[10] ptrace(basis::Meteor.ExactDiag.Basis.AbstractNonSymmetricBasis, v::Union{AbstractArray{T,2} where T, AbstractArray{T,1} where T}, sites::Array{Int64,1}; mode) in Meteor.ExactDiag.Model.Utility at /Users/pidan/Downloads/Meteor.jl-develop/src/ExactDiag/Model/Utility/ptrace.jl:36
[11] ptrace(basis::Meteor.ExactDiag.Basis.AbstractGenericSymmetricBasis, m::MappedMatrix{var"#s177",1} where var"#s177"<:(AbstractArray{T,2} where T), sites::Array{Int64,1}; mode) in Meteor.ExactDiag.Model.Utility at /Users/pidan/Downloads/Meteor.jl-develop/src/ExactDiag/Model/Utility/ptrace.jl:72
[12] ptrace(basis::Meteor.ExactDiag.Basis.AbstractGenericSymmetricBasis, v::Meteor.ExactDiag.Basis.AbstractMappedVector, sites::Array{Int64,1}; mode) in Meteor.ExactDiag.Model.Utility at /Users/pidan/Downloads/Meteor.jl-develop/src/ExactDiag/Model/Utility/ptrace.jl:74
[13] ptrace(model::Meteor.ExactDiag.Model.AbstractModel, sites::Array{Int64,1}) in Meteor.ExactDiag.Model.Utility at /Users/pidan/Downloads/Meteor.jl-develop/src/ExactDiag/Model/Utility/ptrace.jl:81
[14] ptrace(v::AbstractArray{T,1} where T, pos::Array{Int64,1}) in Meteor.QuantumCircuit at /Users/pidan/Downloads/Meteor.jl-develop/src/QuantumCircuit/backends/statevector/def.jl:151